## This python script is for processing the GPS data
#### Problem:
##### The EDA data measure around 5 points per sec while the GPS data measure 1 point per sec. To make the two feature( EDA and GPS) consistent in terms of row number(For better mapping), we need to interpolate the data point of GPS data to match the datapoint from EDA data. 
#### Solution:
##### Calculate average latitude longitude interval of each GPS data point and divide it by 4. Insert 3 additional rows between each two datapoint whose location data is calculated by assuming the speed between each two point is the same. The mathmatic expressions are the following(only the latitude part is shown, longitude part shoud be the same):
###### <center> interval = [Lat<sub>(t+1)</sub>-Lat<sub>(t))</sub>] / 4</center> 
   
######   <center> Lat<sub>(i1)</sub> = Lat<sub>(t)</sub>+  interval</center> 
   
######   <center> Lat<sub>(i2)</sub> = Lat<sub>(t)</sub>+2*interval</center> 
   
######  <center> Lat<sub>(i3)</sub> = Lat<sub>(t)</sub>+3*interval</center> 
######  <center> Lat<sub>(i1，2，3)</sub> is the row inserting between the two GPS datapoint

#### Function:
##### Insert row into the original DataFrame to expand the location data 4 times

In [1]:
import pandas as pd

### Data cleaning

In [2]:
df = pd.read_csv('senseit_1_5_4.csv')

In [7]:
index = (df['number']==0)

In [10]:
df = df[index]

In [16]:
df.columns

Index(['number', 'sound', 'Lat', 'Lon', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')

In [17]:
df.drop(['number','sound','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8'],axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
df.reset_index(inplace = True)

### Setting the index list for matching


In [38]:
indexlist = [i*4 for i in range(553)] #the number in range() should be the same as the total data point

In [33]:
df.drop(['index'],axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
df2 = df.copy()

In [52]:
df2.reset_index(inplace= True)
df2.drop('index',axis=1,inplace =True)

In [56]:
df2.drop(['level_0'],axis=1,inplace = True)

### Shift the data by one row down the calculate the interval

In [64]:
df3 = df2.shift(periods=1)
df2['Interval_lat']=df2['Lat']-df3['Lat']
df2['Interval_lon']=df2['Lon']-df3['Lon']

In [78]:
df2.index = df2.index*4

### Interpolation

In [87]:
for i in indexlist:
    if i != 0:
        #Calculte the interval for latitude and longitude
        inv = df2.loc[i]['Interval_lat']/4 
        inv2= df2.loc[i]['Interval_lon']/4
        # Start from the previous one GPS datapoint to interploate
        Lat = df2.loc[i-4]['Lat']
        Lon = df2.loc[i-4]['Lon']
        # Inserting three datapoint with equal interval
        df2.loc[i-3]=[Lat+inv,Lon+inv,0,0]
        df2.loc[i-2]=[Lat+2*inv,Lon+2*inv,0,0]
        df2.loc[i-1]=[Lat+3*inv,Lon+3*inv,0,0]
        

In [97]:
df2.sort_index(inplace = True)